## Introduction

### The Setup we found to be the best performing one for our dataset uses One vs Rest classifier with Logistic Regression and Bag of Words model for text representation. Below the model is trained before it is served using Docker and Flask. The system uses Jenkins to build a CI CD pipeline which integrates with Git such that everytime code changes or data changes are pushed to the repository, the pipeline execution process will trigger which will end with the deployment of the model. 

## Model Serving Options and the Right Choice
 
### Serving model is one of the important stages in any machine learning project. This is the process of deploying a trained model for the cause it was built for. There are multiple ways of serving a model but we will only be addressing a few of them. The first one that we are going to address is the Tensor flow Serving which has high performance and a flexible system. If we compare this serving model with the consideration mentioned in the lectures, this model mainly deals with the inference aspect of machine learning. It can also store multiple models or versions of the multiple ML models. It also lets you test the model stored on it and its very efficient aswell as the low-overhead implementation adds very minor latency to the model being deployed.

### The second model serving we are going to address is the TorchServe, this is also a very easy and flexible to use system for Pytorch models. Since this model delivers lightweight serving with low latency, the developer or the user gets high performance inference. Here if we compare this model again, with the consideration given in lecture, the system provides  multi-model serving and is able to test different models individually. This system also gives you an option to monitor the health of a server and the whole application in general. RESTful endpoints are also provided for the integration of application. This Supports any machine learning environment including AWS EC2 which can deploy the model on the cloud.

### Finally, the model serving approach we are using for our project is Flask. Since our project is just classifying job tags and wouldn’t really be needing any managing after deployment. Moreover, since in our project we wouldn’t really be needing to parallelise the model inference so we will be going for a traditional way to serve our model which is through Flask, a microweb framework. As explained it does not cover the consideration parts mentioned but its gives more flexibility and more accessible framework for new developers. We also don’t have to worry about defining the protocols and threads. The RESTful API is created through flask through which we can access and use the data in ML model. The interaction is done using GET POST requests. Using Flask a small web application is implemented in python code which runs on a server. In order to potentially deploy it commercially, the built in flask server is not used for development phase but we need to deploy the Flask API on an AWS EC2 server using the boto3 or on GAE using gunicorn.

## Performance of the service and the good and bad points

### Flask is a lightweight web application framework. The strength of Flask lies in its customizability which makes it more flexible for the developer. This gives developer a chance to interoperate with different workflows and systems and also the flexibility to customize their application the way they want. Because of its high scalability it can request any desired amount of data. Another good thing about flask is its quick integration and how there isn’t any strict convention to follow as well. Flask facilitates faster implementations in code and developer can add more features to enrich the product easily. Since its a smaller framework and has fewer layers, it will always do slightly better in performance compared to other systems.
### But the weakness of Flask is that its vulnerable to security risks. It has more complicated maintances for larger project implementation and will cost more for development. Another weakness of Flask is that since it has a singular source which means that it will handle every HTTP request one by one so will take more time. The flexibility can also be taken as weakness since there isn’t any strict convention so might take developer more time figuring out how to organize and design.

In [ ]:
import nltk
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import json
from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import pickle
from scipy import sparse as sp_sparse

import warnings
warnings.filterwarnings('ignore')

np.random.seed(10)

# Below are the functions that will contribute to the pre-processing of the dataset.
def special_characters_removal(text, remove_digits=False):
	pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
	text = re.sub(pattern, '', text)
	return text

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

def create_bow(text, word_ind, dict_size):
    res_vec = np.zeros(dict_size)
    for rec in text.split(' '):
        if rec in word_ind:
            res_vec[word_ind[rec]] +=1
    return res_vec

### Preprocessing of the dataset begins here.

In [ ]:
#Dataset is in json format therefore I shall be using the below method to load the data into the data structure before 
#going further
with open('sampleJobDataWithTags.json', encoding="utf-8") as dataset_json:
	training_dataset = json.load(dataset_json)

preprocessed_dataset = []
lemmatizer = WordNetLemmatizer()

# Data cleansing
for datarecords in training_dataset:
    
    # The BeautifulSoap library helps to scrape data from webpages and provides with the html parser to get the text from 
    # html which is exactly what is being done below. 
	datarecords["title"] = BeautifulSoup(datarecords["title"], "html.parser").get_text()
	datarecords["title"] = unicodedata.normalize('NFKD', datarecords["title"]).encode('ascii', 'ignore').decode('utf-8', 'ignore')
	datarecords["title"]  = special_characters_removal(datarecords["title"])
	datarecords["title"] = datarecords["title"].lower()
    
	datarecords["description"] = BeautifulSoup(datarecords["description"], "html.parser").get_text() 
	datarecords["description"] = unicodedata.normalize('NFKD', datarecords["description"]).encode('ascii', 'ignore').decode('utf-8', 'ignore')    
	datarecords["description"]  = special_characters_removal(datarecords["description"])
	datarecords["description"] = datarecords["description"].lower()
    
    # Performing lemmatization
    # First tokenizing and splitting the data into words before performing lemmatization
	wordlist_title = nltk.word_tokenize(datarecords["title"])
	wordlist_description = nltk.word_tokenize(datarecords["description"])
	datarecords["title"] = ' '.join([lemmatizer.lemmatize(lemw) for lemw in wordlist_title])
	datarecords["description"] = ' '.join([lemmatizer.lemmatize(lemw) for lemw in wordlist_description])

	# appending the data
	preprocessed_dataset.append({"title": datarecords["title"], "description": datarecords["description"], "tags": datarecords["tags"]})

print("Preprocessed Dataset: \n")
print(preprocessed_dataset[0:1])

# Converting json array to dataframe
trainset = pd.read_json(json.dumps(preprocessed_dataset))

#Removing of the Stop Words
stop_words = set(stopwords.words('english'))
trainset['description'] = trainset["description"].apply(lambda t: remove_stopwords(t))

multilb = MultiLabelBinarizer()
multilb.fit(trainset['tags'])

# transform target variable
target = multilb.transform(trainset['tags'])

trainX, testX, ytrain, yval = train_test_split(trainset['description'], target, test_size=0.20, random_state=9)

count_of_words = {}
for desc in trainX:
    for token in desc.split():
        if token not in count_of_words:
            count_of_words[token] = 1
        count_of_words[token] += 1

size_of_dict = 10000
pop_words = sorted(count_of_words, key=count_of_words.get, reverse=True)[:size_of_dict]
words_index = {key: rank for rank, key in enumerate(pop_words, 0)}
index_words = {index:word for word, index in words_index.items()}
everyword = words_index.keys()

trainX_bow = sp_sparse.vstack([sp_sparse.csr_matrix(create_bow(descr, words_index, size_of_dict)) for descr in trainX])
testX_bow = sp_sparse.vstack([sp_sparse.csr_matrix(create_bow(descr, words_index, size_of_dict)) for descr in testX])
print('X_train shape ', trainX_bow.shape, '\nX_val shape ', testX_bow.shape)

### The model training process begins here.

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.multiclass import OneVsRestClassifier

lr = LogisticRegression() 
clf = OneVsRestClassifier(lr)

#fitting the model on training data
clf.fit(trainX_bow, ytrain)

## Serializing the objects

### The trained model along with other pertinent objects needed for the pre-processing of the user inputs on the deployed app are serialized in the script below and saved locally. These saved objects will be loaded before the deployment of the model. The threshold for Logistic Regression is applied after the model is loaded in the 'serve_model.py' script. 

In [ ]:
filename='model.pkl'
pickle.dump(clf, open(filename, 'wb'))

filename='multilb.pkl'
pickle.dump(multilb, open(filename, 'wb'))

filename='wordsindex.pkl'
pickle.dump(words_index, open(filename, 'wb'))

filename='preprocessed_trainset.pkl'
pickle.dump(trainset, open(filename, 'wb'))

## Model Serving, Monitoring and Testing

### The model is wrapped by Flask which is our choice of web framework to build a RESTful API. The 'serve_model' Python script is responsible for receiving application user inputs and generating labels predictions using the locally saved objects. The script after generating the predictions, logs the user inputs and the predicted Job description tags locally in a text file and checks for drift from the respective input and target distributions of the training data. Welsh's t-test has been used for this purpose as it allows for comparison among different sample sizes. The test results in a p-value on which a threshold has been defined to check if the distriubtions are different or the same. 

### Finally, a testing script "test.py" is created to send post requests to the endpoints to check for pre-defined user inputs. The script upon execution will generate a prediction and perform the monitoring of the inputs and the predictions. In the script, a Data Scientist Job description taken from LinkedIn has been defined and another one pertains to Analytics and Client Management, a rather diverse role incorporating knowledge from multiple areas and hence variable tags were expected to be predicted. The model predicts the relevant tags for both the inputs very accurately. Even though, the subsequent model monitoring process described the data distributions as different however that difference doesn't lead to the underperforming of the model. 

In [2]:
#Running this will host the Flask app locally on the port 8080
%run serve_model.py

[nltk_data] Downloading package stopwords to C:\Users\Saud Zaman
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Saud Zaman
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Saud Zaman
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Total Training samples 20000
 * Serving Flask app "serve_model" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader
 * Restarting with windowsapi reloader


## CI CD pipeline (Jenkins, Docker, Github)

### Best industry practices have been followed in the building of CI CD pipeline for the application. This involved setting up a Git repository for the application first and then hooking it to Jenkins where a pipeline is set up for the running of the entire process from building the docker image to running the model training (train_model.py) and model serving (serve_model.py) process. The 'Jenkinsfile' provided with the submission, scripts the stages that are executed in turns, one by one. First stage of the pipeline builds a docker image, followed by the model training and deployment process in the second stage subsequent to which the Flask app with the model is deployed and is accessible on the specified port. Each time, the code changes or data changes are made and pushed to the Git repository, a post request is triggered and is sent to the 'ngrok' Web-hook URL which is mapped to the Jenkins endpoint. Jenkins then takes that as a signal to execute the the entire pipeline. 

### In the process, Docker is used to perform the containerization of the Flask application "model_serve.py" in the system. It runs both files 'train_model.py' and 'model_serve.py' however for the purpose of demonstration under the time constraints, the training command was omitted from the Dockerfile. The 'Dockerfile' that sets out a script which is executed when the image is built using the docker build command. The docker run command launches a container that runs a command specified at the end of the 'Dockerfile' script. This command is the one which triggers the model training and deployment process. The docker run command also specifies the port mapping between the container port where the Flask application is accessible and the Localhost's port. The included Dockerfile defines Ubuntu OS as the container's baseline image and writes code to install the Flask application dependencies that enables it to run in the container. 